In [ ]:
using Dates
using ITensors
using Plots

### 1D chain of Spin-1 particles - Classical Heisenberg Model

A 1D spin chain with open boundary conditions and nearest neighbor interactions for spin-1 particles with coupling strength J is illustrated below.

<img src="./img/spin-1%20chain.png" alt="spin-1 chain" width="50%" style="display: block; margin-left: auto; margin-right: auto;"/>

The model is described by the following Hamiltonian,

$$H = J \sum_{i=1}^{N-1} \vec{S}_i \cdot \vec{S}_{i+1}$$